In [1]:
import pandas as pd
import plotly.express as px

In [11]:
client = pd.read_csv("../data/client.csv")
ePrices = pd.read_csv("../data/electricity_prices.csv")
gasPrices = pd.read_csv("../data/gas_prices.csv")

#weatherPred = pd.read_csv("../data/forecast_weather.csv")
#weatherHist = pd.read_csv("../data/historical_weather.csv")
train = pd.read_csv("../data/train.csv")

# rewrite producing and consuming into a ratio

In [12]:
print(train.shape)
producing = train.loc[train.is_consumption == 0]
consuming = train.loc[train.is_consumption == 1]

(2018352, 9)


In [13]:
train = pd.merge(producing.drop('is_consumption',axis = 1), consuming.drop('is_consumption',axis = 1),on=['data_block_id','prediction_unit_id','datetime','county','is_business','product_type'], how='outer',suffixes=('_prod', '_cons'))
del producing, consuming
print(train.shape)

(1009176, 10)


# merging gas & e prices into train

In [54]:
df_prices = pd.merge(ePrices, gasPrices[['data_block_id','lowest_price_per_mwh','highest_price_per_mwh']], on=['data_block_id'], how='outer',suffixes=('_el', '_gas'))

In [44]:
df_prices['datetime'] = pd.to_datetime(df_prices['forecast_date'])
train['datetime'] = pd.to_datetime(train['datetime'])

df_prices['hour'] = df_prices['datetime'].dt.hour
train['hour'] = train['datetime'].dt.hour

In [45]:
df = pd.merge(train, df_prices,on=['data_block_id','hour'], how='outer',suffixes=('', '_GasEl_prices'))
df = df.drop('datetime_GasEl_prices', axis = 1)

In [52]:
train = df

In [55]:
del df_prices, df

# merging client and train

In [57]:
df = pd.merge(train, client, on=['county','is_business','product_type','data_block_id'], how='inner',suffixes=('', '_client'))
# outer doesn't give more info

In [58]:
df

,county,is_business,product_type,target_prod,datetime,data_block_id,row_id_prod,prediction_unit_id,target_cons,row_id_cons,hour,forecast_date,euros_per_mwh,origin_date,lowest_price_per_mwh,highest_price_per_mwh,eic_count,installed_capacity,date
0,0,0,1,0.793,2021-09-03 00:00:00,2,5856,0,107.129,5857,0,2021-09-02 00:00:00,96.99,2021-09-01 00:00:00,45.62,46.29,108,952.89,2021-09-01
1,0,0,1,2.106,2021-09-03 01:00:00,2,5978,0,81.920,5979,1,2021-09-02 01:00:00,94.77,2021-09-01 01:00:00,45.62,46.29,108,952.89,2021-09-01
2,0,0,1,0.955,2021-09-03 02:00:00,2,6100,0,96.193,6101,2,2021-09-02 02:00:00,93.88,2021-09-01 02:00:00,45.62,46.29,108,952.89,2021-09-01
3,0,0,1,0.711,2021-09-03 03:00:00,2,6222,0,94.536,6223,3,2021-09-02 03:00:00,92.25,2021-09-01 03:00:00,45.62,46.29,108,952.89,2021-09-01
4,0,0,1,0.729,2021-09-03 04:00:00,2,6344,0,99.585,6345,4,2021-09-02 04:00:00,91.83,2021-09-01 04:00:00,45.62,46.29,108,952.89,2021-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004851,15,1,3,111.403,2023-05-31 19:00:00,637,2017830,60,171.252,2017831,19,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,29.00,34.00,55,2188.20,2023-05-29
1004852,15,1,3,43.751,2023-05-31 20:00:00,637,2017960,60,232.759,2017961,20,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,29.00,34.00,55,2188.20,2023-05-29
1004853,15,1,3,7.030,2023-05-31 21:00:00,637,2018090,60,265.328,2018091,21,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,29.00,34.00,55,2188.20,2023-05-29
1004854,15,1,3,0.001,2023-05-31 22:00:00,637,2018220,60,274.569,2018221,22,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,29.00,34.00,55,2188.20,2023-05-29


In [84]:
#feature
df['cust_nr'] = df.groupby('datetime')['datetime'].transform('count')

In [93]:
pd.set_option('display.max_rows', None)
costumer1 = df.loc[df.prediction_unit_id == 67] #.head(100)
print(costumer1.cust_nr.value_counts())
pd.set_option('display.max_rows', 100)

67    4512
66    4008
68    1560
69     672
65     624
64     216
Name: cust_nr, dtype: int64


In [94]:
costumer1['datetime'] = pd.to_datetime(costumer1['datetime'])
fig = px.scatter(costumer1, x='datetime', y=['eic_count','installed_capacity','target','cust_nr'], title='Scatter Plot with Missing Dates')
fig.show()

/var/folders/15/v1vyry112dv3d398dy47k8v40000gn/T/ipykernel_40896/3369716478.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

